## Clean your deployed model endpoint content
-----

#### In this notebook, we will parse through the existing endpoint.json to delete all of the existing endpoints once you are done with running your respective benchmarking tests.

***If you are with running all of the tests, and want to delete the existing endpoints, run this notebook.***

In [2]:
## Import all necessary libraries
import json
import boto3
import logging
from globals import *
from utils import load_config

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [3]:
## Set your logger to display all of the endpoints being cleaned
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

## Load the config file to extract our endpoint.json file and its respective file path
config = load_config(CONFIG_FILE)

In [4]:
## Parse through the existing endpoint.json file
## read the list of deployed endpoints that are active
endpoint_info_list = json.loads(Path(ENDPOINT_LIST_FPATH).read_text())
logger.info(json.dumps(endpoint_info_list, indent=2))

[2024-01-20 20:16:06,475] p371 {885729754.py:4} INFO - [
  {
    "experiment_name": "llama2-70b-g5.48xlarge-huggingface-pytorch-tgi-inference-2.0.1-tgi1.1.0",
    "endpoint": {
      "EndpointName": "llama-2-70b-g5-48xlarge-1705773235",
      "EndpointArn": "arn:aws:sagemaker:us-east-1:015469603702:endpoint/llama-2-70b-g5-48xlarge-1705773235",
      "EndpointConfigName": "llama-2-70b-g5-48xlarge-1705773235",
      "ProductionVariants": [
        {
          "VariantName": "AllTraffic",
          "DeployedImages": [
            {
              "SpecifiedImage": "763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-pytorch-tgi-inference:2.0.1-tgi1.1.0-gpu-py39-cu118-ubuntu20.04",
              "ResolvedImage": "763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-pytorch-tgi-inference@sha256:2739b630b95d8a95e6b4665e66d8243dd43b99c4fdb865feff13aab9c1da06eb",
              "ResolutionTime": "2024-01-20 17:53:58.429000+00:00"
            }
          ],
          "CurrentWeight": 1.0

In [5]:
## initialize a sagemaker client
sm_client = boto3.client("sagemaker")
# Iterate over the endpoint_info_list and mark the items for deletion
for item in endpoint_info_list:   
    
    ## Extract the endpoint name from the deployed model configuration
    ep_name = item['endpoint']["EndpointName"]
    try:
        ## Describe the model endpoint 
        logger.info(f"Going to describing the endpoint -> {ep_name}")
        resp = sm_client.describe_endpoint(EndpointName=ep_name)
        
        ## If the given model endpoint is in service, delete it 
        if resp['EndpointStatus'] == 'InService':
            logger.info(f"going to delete {ep_name}")
            ## deleting the model endpoint
            sm_client.delete_endpoint(EndpointName=ep_name)
            logger.info(f"deleted {ep_name}")
    except Exception as e:
        logger.error(f"error deleting endpoint={ep_name}, exception={e}")

[2024-01-20 20:16:07,996] p371 {707195403.py:10} INFO - Going to describing the endpoint -> llama-2-70b-g5-48xlarge-1705773235
[2024-01-20 20:16:08,174] p371 {707195403.py:15} INFO - going to delete llama-2-70b-g5-48xlarge-1705773235
[2024-01-20 20:16:08,267] p371 {707195403.py:18} INFO - deleted llama-2-70b-g5-48xlarge-1705773235
[2024-01-20 20:16:08,268] p371 {707195403.py:10} INFO - Going to describing the endpoint -> huggingface-pytorch-tgi-inference-2024-01-20-17-53-57-009
[2024-01-20 20:16:08,372] p371 {707195403.py:15} INFO - going to delete huggingface-pytorch-tgi-inference-2024-01-20-17-53-57-009
[2024-01-20 20:16:10,448] p371 {707195403.py:18} INFO - deleted huggingface-pytorch-tgi-inference-2024-01-20-17-53-57-009
